In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from dotenv import load_dotenv
from utils import load_img_path_from_dir, img_to_array, get_train_test_datasets, get_dataset
from preprocess import balance_datasets
from model2 import init as model2_init, compile as model2_compile
from sklearn.model_selection import train_test_split

In [3]:
# get_dataset()

Downloading...
From (uriginal): https://drive.google.com/uc?export=download&id=1Zxpg0eJBkW9T3OA3yoD3mZj5yhI-JqBB
From (redirected): https://drive.google.com/uc?export=download&id=1Zxpg0eJBkW9T3OA3yoD3mZj5yhI-JqBB&confirm=t&uuid=a4055690-8660-4e7d-b282-130127dc7af8
To: /Users/puchi/code/lecorreyann/dog_buster/file.zip
100%|██████████| 160M/160M [00:04<00:00, 37.4MB/s] 


In [3]:
dir_train_0 = os.environ.get('DOGS_DATASET_TRAIN_PATH') + '/0'
dir_train_1 = os.environ.get('DOGS_DATASET_TRAIN_PATH') + '/2'
dir_test_1 = os.environ.get('DOGS_DATASET_TEST_PATH') + '/2'

In [4]:
balance_datasets(
    dir_train_0=dir_train_0,
    dir_train_1=dir_train_1)

🖼 Balance datasets


In [19]:
train_dataset, val_dataset, test_dataset, features_train, features_test = get_train_test_datasets(dir_train_0, dir_train_1, dir_test_1)

Found 600 files belonging to 2 classes.
Found 2 files belonging to 1 classes.


In [22]:
from vgg16 import vgg16,set_non_trainable,add_last_layers,build_model
from utils import *
from preprocess import balance_datasets

In [116]:
model_vgg16 = build_model(num_classes=2)
model_vgg16.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_13 (Flatten)        (None, 25088)             0         
                                                                 
 dense_41 (Dense)            (None, 64)                1605696   
                                                                 
 dense_42 (Dense)            (None, 128)               8320      
                                                                 
 dense_43 (Dense)            (None, 2)                 258       
                                                                 
Total params: 16,328,962
Trainable params: 1,614,274
Non-trainable params: 14,714,688
_________________________________________________________________


In [117]:
es = EarlyStopping(patience=3,restore_best_weights=True)  # Early Stopping normal para la pérdida
history=model_vgg16.fit(train_dataset,validation_data=val_dataset,epochs=20,callbacks=[es])

Epoch 1/20
8/8 [==============================] - 7s 795ms/step - loss: 2.5432 - accuracy: 0.6452 - val_loss: 1.9169 - val_accuracy: 0.6875
Epoch 2/20
8/8 [==============================] - 6s 742ms/step - loss: 0.0349 - accuracy: 0.9839 - val_loss: 0.9341 - val_accuracy: 0.8750
Epoch 3/20
8/8 [==============================] - 6s 745ms/step - loss: 4.5970e-06 - accuracy: 1.0000 - val_loss: 0.5513 - val_accuracy: 0.8750
Epoch 4/20
8/8 [==============================] - 6s 745ms/step - loss: 7.9600e-07 - accuracy: 1.0000 - val_loss: 0.3903 - val_accuracy: 0.8750
Epoch 5/20
8/8 [==============================] - 6s 762ms/step - loss: 3.6916e-07 - accuracy: 1.0000 - val_loss: 0.3088 - val_accuracy: 0.8750
Epoch 6/20
8/8 [==============================] - 6s 774ms/step - loss: 1.9996e-07 - accuracy: 1.0000 - val_loss: 0.2745 - val_accuracy: 0.8750
Epoch 7/20
8/8 [==============================] - 6s 830ms/step - loss: 1.0960e-07 - accuracy: 1.0000 - val_loss: 0.2595 - val_accuracy: 0.8750


In [118]:
model_vgg16.evaluate(test_dataset)

3/3 [==============================] - 2s 556ms/step - loss: 1.8918 - accuracy: 0.8500


[1.8917649984359741, 0.8500000238418579]

In [56]:
img_tests_1 = load_img_path_from_dir(os.environ.get('DATASET_PATH')+ '/1')
predictions_2 = []
for img in img_tests_1:
    img = img_to_array(img)
    predictions_2.append(model_vgg16.predict(img))

1/1 [==============================] - 0s 169ms/step


In [57]:
temp = []
for res in predictions_2:
    dog_id = np.argmax(res)
    temp.append(dog_id)
temp=np.array(temp)

In [58]:
((temp==1).sum())/len(temp)*100

97.34219269102991

## ResNet50

In [7]:
from RESNET import build_resnet
from utils import *
from preprocess import balance_datasets

In [8]:
model_Resnet = build_resnet(num_classes=2)
model_Resnet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 124)               16253052  
                                                                 
 dropout (Dropout)           (None, 124)               0         
                                                                 
 dense_1 (Dense)             (None, 54)                6750      
                                                                 
 dropout_1 (Dropout)         (None, 54)                0         
                                                                 
 dense_2 (Dense)             (None, 2)                 1

In [37]:
es = EarlyStopping(patience=3,restore_best_weights=True)  # Early Stopping normal para la pérdida
history=model_Resnet.fit(train_dataset,validation_data=val_dataset,epochs=20,callbacks=[es])

Epoch 1/20
60/60 [==============================] - 24s 371ms/step - loss: 3.7671 - accuracy: 0.8833 - val_loss: 0.3953 - val_accuracy: 0.9750
Epoch 2/20
60/60 [==============================] - 22s 363ms/step - loss: 0.5442 - accuracy: 0.9771 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 3/20
60/60 [==============================] - 22s 374ms/step - loss: 1.2810 - accuracy: 0.9771 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/20
60/60 [==============================] - 22s 365ms/step - loss: 0.2749 - accuracy: 0.9896 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/20
60/60 [==============================] - 22s 360ms/step - loss: 0.7002 - accuracy: 0.9896 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
60/60 [==============================] - 21s 350ms/step - loss: 0.3597 - accuracy: 0.9937 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [40]:
model_Resnet.evaluate(test_dataset)

1/1 [==============================] - 1s 864ms/step - loss: 172.9913 - accuracy: 0.5000


[172.9912872314453, 0.5]

In [38]:
results_0 = model_Resnet.predict(test_dataset)

1/1 [==============================] - 1s 564ms/step


In [39]:
results_0

array([[0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 5.8248873e-35]], dtype=float32)

In [ ]:
model_Resnet.save(f'models/model_Resnet_1.h5')

# Autoencode

In [12]:
from autoencode import build_autoencoder, build_encoder, build_decoder, compile_autoencoder

In [14]:
encoder = build_encoder(2)
encoder.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 299, 299, 8)       104       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 149, 149, 8)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 148, 148, 16)      528       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 73, 73, 32)        2080      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 36, 36, 32)      

In [15]:
decoder = build_decoder(2)
decoder.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 45000)             135000    
                                                                 
 reshape (Reshape)           (None, 75, 75, 8)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 150, 150, 8)      264       
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 300, 300, 1)      33        
 ranspose)                                                       
                                                                 
Total params: 135,297
Trainable params: 135,297
Non-trainable params: 0
_________________________________________________________________


In [16]:
autoencoder = build_autoencoder(encoder, decoder)
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 sequential_2 (Sequential)   (None, 2)                 85658     
                                                                 
 sequential_3 (Sequential)   (None, 300, 300, 1)       135297    
                                                                 
Total params: 220,955
Trainable params: 220,955
Non-trainable params: 0
_________________________________________________________________


In [17]:
decoder.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 520200)            1560600   
                                                                 
 reshape (Reshape)           (None, 255, 255, 8)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 510, 510, 8)      584       
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 1020, 1020, 1)    73        
 ranspose)                                                       
                                                                 
Total params: 1,561,257
Trainable params: 1,561,257
Non-trainable params: 0
_________________________________________________________________


In [17]:
compile_autoencoder(autoencoder)
autoencoder.fit(features_train, features_train, epochs = 20, batch_size = 32)

Epoch 1/20
19/19 [==============================] - 3s 148ms/step - loss: 15804.1963
Epoch 2/20
19/19 [==============================] - 3s 148ms/step - loss: 15795.5049
Epoch 3/20
19/19 [==============================] - 3s 151ms/step - loss: 15782.0684
Epoch 4/20
19/19 [==============================] - 3s 152ms/step - loss: 15759.7148
Epoch 5/20
19/19 [==============================] - 3s 147ms/step - loss: 15724.2217
Epoch 6/20
19/19 [==============================] - 3s 148ms/step - loss: 15668.7715
Epoch 7/20
19/19 [==============================] - 3s 147ms/step - loss: 15587.8613
Epoch 8/20
19/19 [==============================] - 3s 146ms/step - loss: 15474.6963
Epoch 9/20
19/19 [==============================] - 3s 150ms/step - loss: 15325.3535
Epoch 10/20
19/19 [==============================] - 3s 150ms/step - loss: 15138.8203
Epoch 11/20
19/19 [==============================] - 3s 146ms/step - loss: 14918.8066
Epoch 12/20
19/19 [==============================] - 3s 151ms/s

In [50]:
train_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 255, 255, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [20]:
results = autoencoder.evaluate(features_test)

1/1 [==============================] - 0s 63ms/step - loss: 0.0000e+00


In [21]:
autoencoder.predict(features_test)

1/1 [==============================] - 0s 95ms/step


array([[[[18.975428],
         [19.383984],
         [21.396473],
         ...,
         [19.684076],
         [21.761713],
         [20.34354 ]],

        [[17.761015],
         [16.287497],
         [20.465052],
         ...,
         [16.53945 ],
         [20.810043],
         [18.728533]],

        [[22.743826],
         [21.60946 ],
         [19.82526 ],
         ...,
         [21.939953],
         [20.127625],
         [19.50087 ]],

        ...,

        [[18.006205],
         [16.512701],
         [20.77543 ],
         ...,
         [16.568834],
         [20.85105 ],
         [18.765188]],

        [[23.063255],
         [21.906534],
         [20.09634 ],
         ...,
         [21.980694],
         [20.162855],
         [19.536844]],

        [[21.374943],
         [19.401312],
         [18.079401],
         ...,
         [19.472315],
         [18.14051 ],
         [16.93802 ]]],


       [[[18.975428],
         [19.383984],
         [21.396473],
         ...,
         [19.684

In [63]:
# perro nada que ver
perro_que_no_es = img_to_array(os.environ.get('DOGS_DATASET_TRAIN_PATH') + '/0/n02086240_533.jpg')
perro_que_no_es_encoded = encoder.predict(perro_que_no_es)
perro_que_no_es_encoded

1/1 [==============================] - 0s 15ms/step


array([[ 1., -1.]], dtype=float32)

In [32]:
os.environ.get('DOGS_DATASET_TEST_PATH') + '/2/hihjyihjyihj.jpg'

'./Dataset/Dogs/Test/2/hihjyihjyihj.jpg'

In [61]:
perro_que_es = img_to_array(os.environ.get('DOGS_DATASET_TEST_PATH') + '/2/dehduehdud.jpeg')
perro_que_es = encoder.predict(perro_que_es)

1/1 [==============================] - 0s 15ms/step


In [71]:
perro_que_es

array([[ 1., -1.]], dtype=float32)

In [42]:
from sklearn.metrics import mean_squared_error

In [54]:
def custom_mse(y_true, y_pred):
    # Ensure the vectors are of equal length
    #assert len(y_true) == len(y_pred), "Vectors of unequal length"

    # Calculate the squared differences
    squared_diffs = [(true - pred) ** 2 for true, pred in zip(y_true, y_pred)]

    # Return the mean of the squared differences
    return sum(squared_diffs) / len(y_true)


In [55]:
len(perro_que_no_es)

1

In [56]:
len(perro_que_es)

1

In [57]:
mse = mse = custom_mse(perro_que_no_es[0], perro_que_es[0])
mse

array([[2.37366657e+01, 4.22700005e+01, 6.22166672e+01],
       [1.43466663e+01, 3.01666660e+01, 4.50666656e+01],
       [1.16266670e+01, 2.57666664e+01, 3.94666672e+01],
       [1.23633337e+01, 2.74200001e+01, 3.93133316e+01],
       [1.10166664e+01, 2.61233330e+01, 3.66133347e+01],
       [6.65666676e+00, 1.91233330e+01, 2.73333340e+01],
       [7.04333353e+00, 2.05366669e+01, 2.68833332e+01],
       [9.25666714e+00, 2.47766666e+01, 2.98566666e+01],
       [9.25666714e+00, 2.47766666e+01, 2.98566666e+01],
       [8.17333317e+00, 2.36033325e+01, 2.86033325e+01],
       [8.89666653e+00, 2.60566673e+01, 3.22166672e+01],
       [6.85333347e+00, 2.24533329e+01, 2.81933327e+01],
       [6.99333334e+00, 2.25933342e+01, 2.83333340e+01],
       [9.18666649e+00, 2.65200005e+01, 3.22433319e+01],
       [7.28666687e+00, 2.32333336e+01, 2.90666676e+01],
       [1.32433329e+01, 3.31766663e+01, 3.95533333e+01],
       [1.18199997e+01, 3.08866673e+01, 3.85666656e+01],
       [6.83333349e+00, 2.24333

In [68]:
def mean_squared_error(matrix1, matrix2):
    print(matrix1)
    if matrix1.shape != matrix2.shape:
        raise ValueError("Matrices must have the same shape")

    error = np.mean(np.square(matrix1 - matrix2))
    return error

In [69]:
mean_squared_error(perro_que_no_es, perro_que_es)

[[[[ 56.  76.  93.]
   [ 41.  62.  77.]
   [ 29.  50.  65.]
   ...
   [ 22.  35.  46.]
   [ 47.  73.  87.]
   [ 52.  75.  90.]]

  [[ 63.  83. 100.]
   [ 51.  72.  87.]
   [ 51.  72.  87.]
   ...
   [ 21.  33.  44.]
   [ 60.  83.  97.]
   [ 43.  66.  79.]]

  [[ 46.  67.  82.]
   [ 59.  80.  95.]
   [ 55.  77.  89.]
   ...
   [ 17.  28.  39.]
   [ 65.  87.  99.]
   [ 51.  72.  85.]]

  ...

  [[ 90. 115. 157.]
   [ 91. 116. 158.]
   [100. 129. 170.]
   ...
   [ 88. 146. 207.]
   [ 97. 158. 218.]
   [100. 161. 221.]]

  [[ 92. 117. 159.]
   [ 93. 118. 160.]
   [100. 129. 170.]
   ...
   [ 92. 149. 212.]
   [ 96. 158. 218.]
   [ 97. 159. 219.]]

  [[ 95. 120. 162.]
   [ 94. 119. 161.]
   [ 99. 129. 170.]
   ...
   [ 93. 151. 215.]
   [ 96. 157. 219.]
   [ 96. 157. 219.]]]]


ValueError: Matrices must have the same shape

In [64]:
perro_que_no_es_encoded.shape

(1, 2)

In [66]:
perro_que_es.shape

(1, 2)

In [67]:
perro_que_no_es_encoded.shape == perro_que_es.shape

True

In [1]:
from utils_gaspar import *

In [6]:
print(get_lon_lat())

None


In [8]:
from streamlit_geolocation import streamlit_geolocation
from streamlit_js_eval import streamlit_js_eval, copy_to_clipboard, create_share_link, get_geolocation
import requests

loc = get_geolocation()
print(loc)
if loc and 'coords' in loc:
    latitude, longitude = loc['coords']['latitude'], loc['coords']['longitude']
    location = requests.get(
        f'https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={latitude}&lon={longitude}').json()
print(location['address'])

None


NameError: name 'location' is not defined